In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
num_epochs = 5
batch_size = 40
learning_rate = 0.001
classes = ('plane', 'car' , 'bird',
    'cat', 'deer', 'dog',
    'frog', 'horse', 'ship', 'truck')

cuda


In [2]:
transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize( 
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010) 
    )
])
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def lrp_normalize(model, dataset, std = 0.01):
    optimizer = torch.optim.SGD(model.parameters(), lr = 0.0, momentum=0.9,weight_decay=5e-4)
    criterion = nn.CrossEntropyLoss()

    f2 = lambda w, _=None: softmax2d(normalize_max1(-w)) * len(w[0])
    
    correct, all = 0,0
    for idx, batch in enumerate(tqdm(dataset)):
        img , label = batch[0].cuda(), batch[1].cuda()

#         return img, label
        img.requires_grad = True
        img.retain_grad = True
        
        fake_label = torch.ones_like(label)
        
        output = model(img)
        output_arg = torch.argmax(output, dim=1)
        fake_label = (fake_label + output_arg) % 1000
        
        loss = criterion(output, label)
        loss.backward()
#         torch.sum(output).backward()
        optimizer.zero_grad()
        
        img_lrp = img*img.grad
        img_lrp = f2(img_lrp)
        with torch.no_grad():
            for i in range(len(img_lrp)):
                img_lrp[i] = to_gaussian(img_lrp[i], std = std)
            
            img = img*img_lrp # img_lrp가 음수값인것 지움
            output = model(img)
        
            all += len(label)
            correct += sum(label == torch.argmax(output, dim=1))
        
    print('Accuracy : %.4f' % (correct/all))
    return img, img.grad
    